### 개선판 word2vec 학습

### CBOW 모델 구현

In [1]:
import sys
sys.path.append('..')
from common.np import *  # import numpy as np
from common.layers import Embedding
from ch04.negative_sampling_layer import NegativeSamplingLoss


class CBOW:
    def __init__(self, vocab_size, hidden_size, window_size, corpus):
        V, H = vocab_size, hidden_size

        # 가중치 초기화
        W_in = 0.01 * np.random.randn(V, H).astype('f')
        W_out = 0.01 * np.random.randn(V, H).astype('f')

        # 계층 생성
        self.in_layers = []
        for i in range(2 * window_size):
            layer = Embedding(W_in)  # Embedding 계층 사용
            self.in_layers.append(layer)
        self.ns_loss = NegativeSamplingLoss(W_out, corpus, power=0.75, sample_size=5)

        # 모든 가중치와 기울기를 배열에 모은다.
        layers = self.in_layers + [self.ns_loss]
        self.params, self.grads = [], []
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads

        # 인스턴스 변수에 단어의 분산 표현을 저장한다.
        self.word_vecs = W_in

    def forward(self, contexts, target):
        h = 0
        for i, layer in enumerate(self.in_layers):
            h += layer.forward(contexts[:, i])
        h *= 1 / len(self.in_layers)
        loss = self.ns_loss.forward(h, target)
        return loss

    def backward(self, dout=1):
        dout = self.ns_loss.backward(dout)
        dout *= 1 / len(self.in_layers)
        for layer in self.in_layers:
            layer.backward(dout)
        return None


In [2]:
# coding: utf-8
import sys
sys.path.append('..')  # 부모 디렉터리의 파일을 가져올 수 있도록 설정
from common.layers import *
from ch04.negative_sampling_layer import NegativeSamplingLoss


class SkipGram:
    def __init__(self, vocab_size, hidden_size, window_size, corpus):
        V, H = vocab_size, hidden_size
        rn = np.random.randn

        # 가중치 초기화
        W_in = 0.01 * rn(V, H).astype('f')
        W_out = 0.01 * rn(V, H).astype('f')

        # 계층 생성
        self.in_layer = Embedding(W_in)
        self.loss_layers = []
        for i in range(2 * window_size):
            layer = NegativeSamplingLoss(W_out, corpus, power=0.75, sample_size=5)
            self.loss_layers.append(layer)

        # 모든 가중치와 기울기를 리스트에 모은다.
        layers = [self.in_layer] + self.loss_layers
        self.params, self.grads = [], []
        for layer in layers:
            self.params += layer.params
            self.grads += layer.grads

        # 인스턴스 변수에 단어의 분산 표현을 저장한다.
        self.word_vecs = W_in

    def forward(self, contexts, target):
        h = self.in_layer.forward(target)

        loss = 0
        for i, layer in enumerate(self.loss_layers):
            loss += layer.forward(h, contexts[:, i])
        return loss

    def backward(self, dout=1):
        dh = 0
        for i, layer in enumerate(self.loss_layers):
            dh += layer.backward(dout)
        self.in_layer.backward(dh)
        return None


### CBOW 모델 학습 코드

In [ ]:
# coding: utf-8
import sys
sys.path.append('..')
import numpy as np
from common import config
# GPU에서 실행하려면 아래 주석을 해제하세요(CuPy 필요).
# ===============================================
# config.GPU = True
# ===============================================
import pickle
from common.trainer import Trainer
from common.optimizer import Adam
from cbow import CBOW
from skip_gram import SkipGram
from common.util import create_contexts_target, to_cpu, to_gpu
from dataset import ptb


# 하이퍼파라미터 설정
window_size = 5
hidden_size = 100
batch_size = 100
max_epoch = 10

# 데이터 읽기
corpus, word_to_id, id_to_word = ptb.load_data('train')
vocab_size = len(word_to_id)

contexts, target = create_contexts_target(corpus, window_size)
if config.GPU:
    contexts, target = to_gpu(contexts), to_gpu(target)

# 모델 등 생성
model = CBOW(vocab_size, hidden_size, window_size, corpus)
# model = SkipGram(vocab_size, hidden_size, window_size, corpus)
optimizer = Adam()
trainer = Trainer(model, optimizer)

# 학습 시작
trainer.fit(contexts, target, max_epoch, batch_size)
trainer.plot()

# 나중에 사용할 수 있도록 필요한 데이터 저장
word_vecs = model.word_vecs
if config.GPU:
    word_vecs = to_cpu(word_vecs)
params = {}
params['word_vecs'] = word_vecs.astype(np.float16)
params['word_to_id'] = word_to_id
params['id_to_word'] = id_to_word
pkl_file = 'cbow_params.pkl'  # or 'skipgram_params.pkl'
with open(pkl_file, 'wb') as f:
    pickle.dump(params, f, -1)


| 에폭 1 |  반복 1 / 9295 | 시간 0[s] | 손실 4.16
| 에폭 1 |  반복 21 / 9295 | 시간 2[s] | 손실 4.16
| 에폭 1 |  반복 41 / 9295 | 시간 5[s] | 손실 4.15
| 에폭 1 |  반복 61 / 9295 | 시간 7[s] | 손실 4.12
| 에폭 1 |  반복 81 / 9295 | 시간 10[s] | 손실 4.04
| 에폭 1 |  반복 101 / 9295 | 시간 13[s] | 손실 3.92
| 에폭 1 |  반복 121 / 9295 | 시간 16[s] | 손실 3.77
| 에폭 1 |  반복 141 / 9295 | 시간 19[s] | 손실 3.62
| 에폭 1 |  반복 161 / 9295 | 시간 22[s] | 손실 3.49
| 에폭 1 |  반복 181 / 9295 | 시간 25[s] | 손실 3.37
| 에폭 1 |  반복 201 / 9295 | 시간 28[s] | 손실 3.25
| 에폭 1 |  반복 221 / 9295 | 시간 31[s] | 손실 3.15
| 에폭 1 |  반복 241 / 9295 | 시간 33[s] | 손실 3.08
| 에폭 1 |  반복 261 / 9295 | 시간 36[s] | 손실 3.01
| 에폭 1 |  반복 281 / 9295 | 시간 40[s] | 손실 2.94
| 에폭 1 |  반복 301 / 9295 | 시간 42[s] | 손실 2.93
| 에폭 1 |  반복 321 / 9295 | 시간 45[s] | 손실 2.84
| 에폭 1 |  반복 341 / 9295 | 시간 48[s] | 손실 2.83
| 에폭 1 |  반복 361 / 9295 | 시간 52[s] | 손실 2.83
| 에폭 1 |  반복 381 / 9295 | 시간 54[s] | 손실 2.80
| 에폭 1 |  반복 401 / 9295 | 시간 57[s] | 손실 2.75
| 에폭 1 |  반복 421 / 9295 | 시간 60[s] | 손실 2.75
| 에폭 1 |  반복 441 / 9

###  CBOW 모델 평가

In [ ]:
# coding: utf-8
import sys
sys.path.append('..')
from common.util import most_similar, analogy
import pickle


pkl_file = 'cbow_params.pkl'
# pkl_file = 'skipgram_params.pkl'

with open(pkl_file, 'rb') as f:
    params = pickle.load(f)
    word_vecs = params['word_vecs']
    word_to_id = params['word_to_id']
    id_to_word = params['id_to_word']

# 가장 비슷한(most similar) 단어 뽑기
querys = ['you', 'year', 'car', 'toyota']
for query in querys:
    most_similar(query, word_to_id, id_to_word, word_vecs, top=5)

In [35]:
# 유추(analogy) 작업
print('-'*50)
analogy('king', 'man', 'queen',  word_to_id, id_to_word, word_vecs)
analogy('take', 'took', 'go',  word_to_id, id_to_word, word_vecs)
analogy('car', 'cars', 'child',  word_to_id, id_to_word, word_vecs)
analogy('good', 'better', 'bad',  word_to_id, id_to_word, word_vecs)

--------------------------------------------------

[analogy] king:man = queen:?
 a.m: 5.7421875
 woman: 5.50390625
 naczelnik: 5.10546875
 carolinas: 4.93359375
 hacker: 4.74609375

[analogy] take:took = go:?
 went: 4.6796875
 're: 4.45703125
 came: 4.4140625
 were: 4.07421875
 was: 4.0625

[analogy] car:cars = child:?
 a.m: 6.75390625
 rape: 5.875
 children: 5.3203125
 daffynition: 4.734375
 women: 4.5390625

[analogy] good:better = bad:?
 rather: 5.78125
 more: 5.73828125
 less: 5.40625
 greater: 4.5078125
 fewer: 4.1328125
